In [1]:
import json.decoder

import numpy as np
import pandas as pd
import sklearn as sk
import json as json
import matplotlib
import pickle as pk
import matplotlib.pyplot as plt
from IPython.display import display

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('scikit-learn version:', sk.__version__)
print('json version:', json.__version__)
print('matplotlib version:', matplotlib.__version__)

numpy version: 1.23.3
pandas version: 1.5.0
scikit-learn version: 1.1.2
json version: 2.0.9
matplotlib version: 3.6.0


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer

def strToList(list_l, splitSymbol):
    list_l = list_l.split(splitSymbol)
    temp = list()
    for l in list_l:
        l = l.replace("[",'').replace("]",'').replace("'", '').replace(" ", '')
        temp.append(l)
    return temp

class ingredientsToList(BaseEstimator, TransformerMixin):
    def __init__(self, columns = []):
        self.columns = columns
    def fit(self, X):
        return self
    def transform(self, X):
      for column in self.columns:
        X[column] = X[column].apply(lambda x : strToList(x, ','))
      return X

class ingredientsToOneHot(BaseEstimator, TransformerMixin):
    def __init__(self, columns = []):
        self.columns = columns
    def fit(self, X):
        return self
    def transform(self, X):
        cv = CountVectorizer(analyzer=lambda x: x)
        for column in self.columns:
            test = cv.fit_transform(X[column].to_list())
            test_columns = [x for x in cv.get_feature_names()]
            X = X.join(pd.DataFrame(test.toarray(), columns = test_columns, index = X.index))
            #X = X.join(pd.DataFrame(test.toarray(), index = X.index))
        return X

class nutritionDataIntoCol(BaseEstimator, TransformerMixin):
    def fit(self, X):
        return self

    def transform(self, X):
      nutrition_X = pd.DataFrame(X['nutrition'].to_list(), columns = ['calories', 'total fat', 'sugar_nutrition', 'sodium', 'protein', 'saturated fat', 'carbohydrates'], index = X.index)

      nutrition_X_col = nutrition_X.columns
      for col in nutrition_X_col:
        nutrition_X[col] = nutrition_X[col].apply(lambda x: float(x))

      X = X.join(nutrition_X)
      return X

class getFeatureColumns(BaseEstimator, TransformerMixin):
    def fit(self, X):
        return self
    def transform(self, X):
        col = list(X.columns)
        for c in ['ingredients', 'nutrition', 'n_steps', 'n_ingredients']:
            col.remove(c)
        return X[col]

In [ ]:
from sklearn.pipeline import Pipeline

pip = Pipeline([
    ('ingredientsToList', ingredientsToList(columns = ['ingredients', 'nutrition'])),
    ('ingredientToOneHotColumns', ingredientsToOneHot(columns = ['ingredients'])),
    ('nutritionData', nutritionDataIntoCol()),
    ('getFeatureColumns', getFeatureColumns())
])
all_withFeatures = pip.transform(train_test)
all_withFeatures.head()


Taking of rare ingredients

In [ ]:
col = list(all_withFeatures.columns)
for column in ['rating', 'calories', 'total fat', 'sugar_nutrition', 'sodium', 'protein',
       'saturated fat', 'carbohydrates']:
    col.remove(column)
sum_ingredients = pd.DataFrame(all_withFeatures[col].sum(axis = 0)/all_withFeatures.shape[0])

sns.set(style = "whitegrid")
ax = sns.boxenplot(x = sum_ingredients.values*100)
ax.set_xticks(np.arange(0,10, 2))
ax.set_xlabel('Number of time an ingredient was used')
plt.show()

sum_freq_ingred_index = list(sum_ingredients[sum_ingredients[0]>0.01].index)